In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
import datetime
import os

# 获取数据
## 缺失值处理

In [2]:
def get_processed_data():
    dataset1 = pd.read_csv('data_preprocessed_2/ProcessDataSet1.csv')
    dataset2 = pd.read_csv('data_preprocessed_2/ProcessDataSet2.csv')
    dataset3 = pd.read_csv('data_preprocessed_2/ProcessDataSet3.csv')

    dataset1.drop_duplicates(inplace=True)
    dataset2.drop_duplicates(inplace=True)
    dataset3.drop_duplicates(inplace=True)

    dataset12 = pd.concat([dataset1, dataset2], axis=0)

    dataset12.fillna(0, inplace=True)
    dataset3.fillna(0, inplace=True)
    return dataset12, dataset3

# 模型训练
## 训练集的处理

In [3]:
def train_xgb(dataset12, dataset3):
    predict_dataset = dataset3[['User_id', 'Coupon_id', 'Date_received']].copy()
    predict_dataset.Date_received = pd.to_datetime(predict_dataset.Date_received, format='%Y-%m-%d')
    predict_dataset.Date_received = predict_dataset.Date_received.dt.strftime('%Y%m%d')

    # 将数据转化为dmatric格式
    dataset12_x = dataset12.drop(
        columns=['User_id', 'Merchant_id', 'Discount_rate', 'Date_received', 'discount_rate_x', 'discount_rate_y',
                 'Date', 'Coupon_id', 'label'], axis=1)
    dataset3_x = dataset3.drop(
        columns=['User_id', 'Merchant_id', 'Discount_rate', 'Date_received', 'discount_rate_x', 'discount_rate_y',
                 'Coupon_id'], axis=1)

    train_dmatrix = xgb.DMatrix(dataset12_x, label=dataset12.label)
    predict_dmatrix = xgb.DMatrix(dataset3_x)
    params = {'booster': 'gbtree',
              'objective': 'binary:logistic',
              'eval_metric': 'auc',
              'gamma': 0.1,
              'min_child_weight': 1.1,
              'max_depth': 5,
              'lambda': 10,
              'subsample': 0.7,
              'colsample_bytree': 0.7,
              'colsample_bylevel': 0.7,
              'eta': 0.01,
              'tree_method': 'gpu_hist',
              'seed': 0,
              'nthread': cpu_jobs,
              'predictor': 'cpu_predictor'
              }

    # 使用xgb.cv优化num_boost_round参数
    cvresult = xgb.cv(params, train_dmatrix, num_boost_round=10000, nfold=2, metrics='auc', seed=0, callbacks=[
        xgb.callback.print_evaluation(show_stdv=False),
        xgb.callback.early_stop(50)
    ])
    num_round_best = cvresult.shape[0] - 1
    print('Best round num: ', num_round_best)

    # 使用优化后的num_boost_round参数训练模型
    watchlist = [(train_dmatrix, 'train')]
    model = xgb.train(params, train_dmatrix, num_boost_round=num_round_best, evals=watchlist)

    model.save_model('train_dir_2/xgbmodel_cv_new')
    params['predictor'] = 'cpu_predictor'
    model_cv = xgb.Booster(params)
    model_cv.load_model('train_dir_2/xgbmodel_cv_new')

    # predict test set
    dataset3_predict = predict_dataset.copy()
    dataset3_predict['label'] = model_cv.predict(predict_dmatrix)

    # 标签归一化
    dataset3_predict.label = MinMaxScaler(copy=True, feature_range=(0, 1)).fit_transform(
        dataset3_predict.label.values.reshape(-1, 1))
    dataset3_predict.sort_values(by=['Coupon_id', 'label'], inplace=True)
    dataset3_predict.to_csv("train_dir_2/xgb_cv_preds.csv", index=None, header=None)
    print(dataset3_predict.describe())

    # 在dataset12上计算auc
    # model = xgb.Booster()
    # model.load_model('train_dir_2/xgbmodel')

    temp = dataset12[['Coupon_id', 'label']].copy()
    temp['pred'] = model.predict(xgb.DMatrix(dataset12_x))
    temp.pred = MinMaxScaler(copy=True, feature_range=(0, 1)).fit_transform(temp['pred'].values.reshape(-1, 1))
    print(myauc(temp))

# 性能评价函数

In [4]:
# 性能评价函数
def myauc(test):
    testgroup = test.groupby(['Coupon_id'])
    aucs = []
    for i in testgroup:
        tmpdf = i[1]
        if len(tmpdf['label'].unique()) != 2:
            continue
        fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred'], pos_label=1)
        aucs.append(auc(fpr, tpr))
    return np.average(aucs)

# 模型训练并保存-0.7983

In [ ]:
# 获取数据
dataset12, dataset3 = get_processed_data()

In [6]:
dataset12.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_rate_x,discount_rate_y,discount_rate,...,on_u4,on_u5,on_u6,on_u7,on_u8,on_u9,on_u10,on_u11,on_u12,on_u13
0,1832624,3381,7610,200:20,0,2016-04-29,1970-01-01,200.0,20.0,0.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,163606,1569,5054,200:30,10,2016-04-21,1970-01-01,200.0,30.0,0.850000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1113008,1361,11166,20:1,0,2016-05-15,2016-05-21,20.0,1.0,0.950000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4061024,3381,7610,200:20,10,2016-04-26,1970-01-01,200.0,20.0,0.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,106443,450,3732,30:5,11,2016-04-29,1970-01-01,30.0,5.0,0.833333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
dataset12.shape

(383386, 124)

In [9]:
dataset12.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383386 entries, 0 to 252585
Columns: 124 entries, User_id to on_u13
dtypes: float64(96), int64(25), object(3)
memory usage: 375.6+ MB


In [11]:
dataset12.describe()

,User_id,Merchant_id,Coupon_id,Distance,discount_rate_x,discount_rate_y,discount_rate,label,weekday,day,...,on_u4,on_u5,on_u6,on_u7,on_u8,on_u9,on_u10,on_u11,on_u12,on_u13
count,3.833860e+05,383386.000000,383386.000000,383386.000000,383386.000000,383386.000000,383386.000000,383386.000000,383386.000000,383386.000000,...,383386.000000,383386.000000,383386.000000,383386.000000,383386.0,383386.000000,383386.000000,383386.0,383386.000000,383386.000000
mean,3.683603e+06,3653.920730,6287.911630,3.216938,58.047151,8.606796,0.838597,0.082546,3.086717,17.428612,...,0.538374,0.165142,0.078388,0.019834,0.0,0.078388,0.041178,0.0,0.001983,0.001983
std,2.123219e+06,2577.836469,3938.971496,4.154925,59.771475,8.860719,0.092783,0.275195,1.984455,8.455349,...,1.979329,0.355102,0.502537,0.119940,0.0,0.502537,0.198702,0.0,0.032386,0.032386
min,4.000000e+00,2.000000,1.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
25%,1.843305e+06,1244.000000,2418.000000,0.000000,20.000000,5.000000,0.800000,0.000000,1.000000,11.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
50%,3.685242e+06,3381.000000,5584.000000,1.000000,30.000000,5.000000,0.833333,0.000000,3.000000,19.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
75%,5.522414e+06,5803.000000,9566.000000,6.000000,100.000000,10.000000,0.900000,0.000000,5.000000,24.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
max,7.360961e+06,8856.000000,14045.000000,11.000000,300.000000,100.000000,0.990000,1.000000,6.000000,31.000000,...,319.000000,1.000000,64.000000,1.000000,0.0,64.000000,1.000000,0.0,0.985915,0.985915


In [12]:
print([column for column in dataset12])

['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance', 'Date_received', 'Date', 'discount_rate_x', 'discount_rate_y', 'discount_rate', 'label', 'weekday', 'day', 'u2', 'u3', 'u19', 'u1', 'u4', 'u5', 'u25', 'u20', 'u6', 'u7', 'u8', 'u9', 'u10', 'u11', 'u21', 'u22', 'u23', 'u24', 'u45', 'u27', 'u28', 'u32', 'u47', 'u33', 'u34', 'u35', 'u36', 'u37', 'discount_type', 'u41', 'u42', 'u43', 'u44', 'u48', 'u49', 'm0', 'm1', 'm2', 'm3', 'm4', 'm7', 'm5', 'm6', 'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15', 'm18', 'm19', 'm20', 'm21', 'm22', 'm23', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c8', 'c9', 'c10', 'c11', 'c12', 'um1', 'um2', 'um3', 'um4', 'um5', 'um6', 'um7', 'um8', 'um9', 'um10', 'um11', 'um12', 'o1', 'o2', 'o17', 'o18', 'o3', 'o4', 'o5', 'o6', 'o7', 'o8', 'o9', 'o10', 'o11', 'o12', 'o13', 'o14', 'o15', 'o16', 'on_u1', 'on_u2', 'on_u3', 'on_u4', 'on_u5', 'on_u6', 'on_u7', 'on_u8', 'on_u9', 'on_u10', 'on_u11', 'on_u12', 'on_u13']


In [10]:
dataset3.shape

(112803, 122)

In [5]:
start = datetime.datetime.now()
print(start.strftime('%Y-%m-%d %H:%M:%S'))
cpu_jobs = os.cpu_count() - 1
date_null = pd.to_datetime('1970-01-01', format='%Y-%m-%d')

dataset12, dataset3 = get_processed_data()
train_xgb(dataset12, dataset3)

print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print('time costed is: %s s' % (datetime.datetime.now() - start).seconds)

2020-03-06 13:07:29
[0]	train-auc:0.82972	test-auc:0.82835
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 50 rounds.
[1]	train-auc:0.83615	test-auc:0.83513
[2]	train-auc:0.84745	test-auc:0.84609
[3]	train-auc:0.85015	test-auc:0.84850
[4]	train-auc:0.85415	test-auc:0.85298
[5]	train-auc:0.85444	test-auc:0.85324
[6]	train-auc:0.85859	test-auc:0.85735
[7]	train-auc:0.86065	test-auc:0.85951
[8]	train-auc:0.86068	test-auc:0.85961
[9]	train-auc:0.86198	test-auc:0.86090
[10]	train-auc:0.86290	test-auc:0.86175
[11]	train-auc:0.86382	test-auc:0.86284
[12]	train-auc:0.86326	test-auc:0.86229
[13]	train-auc:0.86336	test-auc:0.86240
[14]	train-auc:0.86339	test-auc:0.86243
[15]	train-auc:0.86359	test-auc:0.86252
[16]	train-auc:0.86345	test-auc:0.86209
[17]	train-auc:0.86317	test-auc:0.86170
[18]	train-auc:0.86323	test-auc:0.86172
[19]	train-auc:0.86325	test-auc:0.86177
[20]	train-auc:0.86362	test-auc:0.86213
[21]	trai

[199]	train-auc:0.87883	test-auc:0.87684
[200]	train-auc:0.87888	test-auc:0.87689
[201]	train-auc:0.87907	test-auc:0.87708
[202]	train-auc:0.87920	test-auc:0.87719
[203]	train-auc:0.87932	test-auc:0.87730
[204]	train-auc:0.87940	test-auc:0.87738
[205]	train-auc:0.87948	test-auc:0.87746
[206]	train-auc:0.87957	test-auc:0.87755
[207]	train-auc:0.87953	test-auc:0.87752
[208]	train-auc:0.87956	test-auc:0.87755
[209]	train-auc:0.87965	test-auc:0.87762
[210]	train-auc:0.87964	test-auc:0.87760
[211]	train-auc:0.87974	test-auc:0.87768
[212]	train-auc:0.87984	test-auc:0.87778
[213]	train-auc:0.87983	test-auc:0.87776
[214]	train-auc:0.87996	test-auc:0.87790
[215]	train-auc:0.87999	test-auc:0.87792
[216]	train-auc:0.88005	test-auc:0.87797
[217]	train-auc:0.88006	test-auc:0.87799
[218]	train-auc:0.88022	test-auc:0.87814
[219]	train-auc:0.88029	test-auc:0.87820
[220]	train-auc:0.88031	test-auc:0.87820
[221]	train-auc:0.88046	test-auc:0.87835
[222]	train-auc:0.88055	test-auc:0.87844
[223]	train-auc:

[399]	train-auc:0.88882	test-auc:0.88551
[400]	train-auc:0.88885	test-auc:0.88553
[401]	train-auc:0.88889	test-auc:0.88558
[402]	train-auc:0.88891	test-auc:0.88559
[403]	train-auc:0.88895	test-auc:0.88562
[404]	train-auc:0.88898	test-auc:0.88565
[405]	train-auc:0.88902	test-auc:0.88567
[406]	train-auc:0.88906	test-auc:0.88572
[407]	train-auc:0.88911	test-auc:0.88576
[408]	train-auc:0.88916	test-auc:0.88581
[409]	train-auc:0.88922	test-auc:0.88585
[410]	train-auc:0.88925	test-auc:0.88587
[411]	train-auc:0.88928	test-auc:0.88591
[412]	train-auc:0.88932	test-auc:0.88594
[413]	train-auc:0.88936	test-auc:0.88598
[414]	train-auc:0.88941	test-auc:0.88602
[415]	train-auc:0.88946	test-auc:0.88606
[416]	train-auc:0.88952	test-auc:0.88612
[417]	train-auc:0.88956	test-auc:0.88615
[418]	train-auc:0.88959	test-auc:0.88617
[419]	train-auc:0.88963	test-auc:0.88620
[420]	train-auc:0.88968	test-auc:0.88624
[421]	train-auc:0.88970	test-auc:0.88626
[422]	train-auc:0.88975	test-auc:0.88630
[423]	train-auc:

[599]	train-auc:0.89511	test-auc:0.89059
[600]	train-auc:0.89513	test-auc:0.89061
[601]	train-auc:0.89515	test-auc:0.89062
[602]	train-auc:0.89517	test-auc:0.89064
[603]	train-auc:0.89519	test-auc:0.89065
[604]	train-auc:0.89521	test-auc:0.89067
[605]	train-auc:0.89525	test-auc:0.89070
[606]	train-auc:0.89527	test-auc:0.89072
[607]	train-auc:0.89530	test-auc:0.89074
[608]	train-auc:0.89533	test-auc:0.89076
[609]	train-auc:0.89535	test-auc:0.89078
[610]	train-auc:0.89536	test-auc:0.89079
[611]	train-auc:0.89538	test-auc:0.89081
[612]	train-auc:0.89540	test-auc:0.89082
[613]	train-auc:0.89542	test-auc:0.89083
[614]	train-auc:0.89544	test-auc:0.89085
[615]	train-auc:0.89547	test-auc:0.89087
[616]	train-auc:0.89550	test-auc:0.89090
[617]	train-auc:0.89552	test-auc:0.89092
[618]	train-auc:0.89555	test-auc:0.89094
[619]	train-auc:0.89557	test-auc:0.89095
[620]	train-auc:0.89559	test-auc:0.89098
[621]	train-auc:0.89560	test-auc:0.89098
[622]	train-auc:0.89563	test-auc:0.89101
[623]	train-auc:

[799]	train-auc:0.89919	test-auc:0.89354
[800]	train-auc:0.89921	test-auc:0.89356
[801]	train-auc:0.89925	test-auc:0.89358
[802]	train-auc:0.89928	test-auc:0.89361
[803]	train-auc:0.89930	test-auc:0.89362
[804]	train-auc:0.89932	test-auc:0.89364
[805]	train-auc:0.89933	test-auc:0.89365
[806]	train-auc:0.89935	test-auc:0.89366
[807]	train-auc:0.89937	test-auc:0.89367
[808]	train-auc:0.89939	test-auc:0.89369
[809]	train-auc:0.89941	test-auc:0.89370
[810]	train-auc:0.89943	test-auc:0.89372
[811]	train-auc:0.89945	test-auc:0.89372
[812]	train-auc:0.89945	test-auc:0.89373
[813]	train-auc:0.89947	test-auc:0.89374
[814]	train-auc:0.89948	test-auc:0.89375
[815]	train-auc:0.89949	test-auc:0.89375
[816]	train-auc:0.89951	test-auc:0.89376
[817]	train-auc:0.89953	test-auc:0.89378
[818]	train-auc:0.89954	test-auc:0.89379
[819]	train-auc:0.89955	test-auc:0.89380
[820]	train-auc:0.89956	test-auc:0.89380
[821]	train-auc:0.89959	test-auc:0.89382
[822]	train-auc:0.89960	test-auc:0.89383
[823]	train-auc:

...

[4771]	train-auc:0.91829
[4772]	train-auc:0.91829
[4773]	train-auc:0.91829
[4774]	train-auc:0.91829
[4775]	train-auc:0.91830
[4776]	train-auc:0.91830
[4777]	train-auc:0.91830
[4778]	train-auc:0.91830
[4779]	train-auc:0.91831
[4780]	train-auc:0.91831
[4781]	train-auc:0.91831
[4782]	train-auc:0.91831
[4783]	train-auc:0.91832
[4784]	train-auc:0.91832
[4785]	train-auc:0.91832
[4786]	train-auc:0.91833
[4787]	train-auc:0.91833
[4788]	train-auc:0.91833
[4789]	train-auc:0.91833
[4790]	train-auc:0.91834
[4791]	train-auc:0.91834
[4792]	train-auc:0.91834
[4793]	train-auc:0.91834
[4794]	train-auc:0.91834
[4795]	train-auc:0.91835
[4796]	train-auc:0.91835
[4797]	train-auc:0.91835
[4798]	train-auc:0.91835
[4799]	train-auc:0.91836
[4800]	train-auc:0.91836
[4801]	train-auc:0.91836
[4802]	train-auc:0.91836
[4803]	train-auc:0.91837
[4804]	train-auc:0.91837
[4805]	train-auc:0.91838
[4806]	train-auc:0.91838
[4807]	train-auc:0.91838
[4808]	train-auc:0.91838
[4809]	train-auc:0.91839
[4810]	train-auc:0.91839


[5099]	train-auc:0.91922
[5100]	train-auc:0.91922
[5101]	train-auc:0.91922
[5102]	train-auc:0.91922
[5103]	train-auc:0.91922
[5104]	train-auc:0.91923
[5105]	train-auc:0.91923
[5106]	train-auc:0.91923
[5107]	train-auc:0.91924
[5108]	train-auc:0.91924
[5109]	train-auc:0.91924
[5110]	train-auc:0.91925
[5111]	train-auc:0.91925
[5112]	train-auc:0.91925
[5113]	train-auc:0.91925
[5114]	train-auc:0.91926
[5115]	train-auc:0.91926
[5116]	train-auc:0.91926
[5117]	train-auc:0.91926
[5118]	train-auc:0.91926
[5119]	train-auc:0.91927
[5120]	train-auc:0.91927
[5121]	train-auc:0.91927
[5122]	train-auc:0.91928
[5123]	train-auc:0.91928
[5124]	train-auc:0.91928
[5125]	train-auc:0.91929
[5126]	train-auc:0.91929
[5127]	train-auc:0.91929
[5128]	train-auc:0.91930
[5129]	train-auc:0.91930
[5130]	train-auc:0.91930
[5131]	train-auc:0.91931
[5132]	train-auc:0.91931
[5133]	train-auc:0.91931
[5134]	train-auc:0.91932
[5135]	train-auc:0.91932
[5136]	train-auc:0.91932
[5137]	train-auc:0.91932
[5138]	train-auc:0.91933


[5427]	train-auc:0.92004
[5428]	train-auc:0.92004
[5429]	train-auc:0.92005
[5430]	train-auc:0.92005
[5431]	train-auc:0.92005
[5432]	train-auc:0.92005
[5433]	train-auc:0.92005
[5434]	train-auc:0.92006
[5435]	train-auc:0.92006
[5436]	train-auc:0.92006
[5437]	train-auc:0.92006
[5438]	train-auc:0.92007
[5439]	train-auc:0.92007
[5440]	train-auc:0.92007
[5441]	train-auc:0.92007
[5442]	train-auc:0.92008
[5443]	train-auc:0.92008
[5444]	train-auc:0.92009
[5445]	train-auc:0.92009
[5446]	train-auc:0.92009
[5447]	train-auc:0.92009
[5448]	train-auc:0.92010
[5449]	train-auc:0.92010
[5450]	train-auc:0.92010
[5451]	train-auc:0.92010
[5452]	train-auc:0.92010
[5453]	train-auc:0.92011
[5454]	train-auc:0.92011
[5455]	train-auc:0.92011
[5456]	train-auc:0.92011
[5457]	train-auc:0.92012
[5458]	train-auc:0.92012
[5459]	train-auc:0.92012
[5460]	train-auc:0.92012
[5461]	train-auc:0.92013
[5462]	train-auc:0.92013
[5463]	train-auc:0.92013
[5464]	train-auc:0.92014
[5465]	train-auc:0.92014
[5466]	train-auc:0.92014


[5755]	train-auc:0.92088
[5756]	train-auc:0.92088
[5757]	train-auc:0.92089
[5758]	train-auc:0.92089
[5759]	train-auc:0.92089
[5760]	train-auc:0.92089
[5761]	train-auc:0.92090
[5762]	train-auc:0.92090
[5763]	train-auc:0.92090
[5764]	train-auc:0.92091
[5765]	train-auc:0.92091
[5766]	train-auc:0.92092
[5767]	train-auc:0.92092
[5768]	train-auc:0.92092
[5769]	train-auc:0.92092
[5770]	train-auc:0.92093
[5771]	train-auc:0.92093
[5772]	train-auc:0.92093
[5773]	train-auc:0.92093
[5774]	train-auc:0.92094
[5775]	train-auc:0.92094
[5776]	train-auc:0.92094
[5777]	train-auc:0.92094
[5778]	train-auc:0.92095
[5779]	train-auc:0.92095
[5780]	train-auc:0.92095
[5781]	train-auc:0.92095
[5782]	train-auc:0.92096
[5783]	train-auc:0.92096
[5784]	train-auc:0.92096
[5785]	train-auc:0.92096
[5786]	train-auc:0.92096
[5787]	train-auc:0.92097
[5788]	train-auc:0.92097
[5789]	train-auc:0.92097
[5790]	train-auc:0.92097
[5791]	train-auc:0.92098
[5792]	train-auc:0.92098
[5793]	train-auc:0.92098
[5794]	train-auc:0.92099


[6083]	train-auc:0.92170
[6084]	train-auc:0.92171
[6085]	train-auc:0.92171
[6086]	train-auc:0.92171
[6087]	train-auc:0.92171
[6088]	train-auc:0.92171
[6089]	train-auc:0.92172
[6090]	train-auc:0.92172
[6091]	train-auc:0.92172
[6092]	train-auc:0.92172
[6093]	train-auc:0.92173
[6094]	train-auc:0.92173
[6095]	train-auc:0.92173
[6096]	train-auc:0.92174
[6097]	train-auc:0.92174
[6098]	train-auc:0.92174
[6099]	train-auc:0.92174
[6100]	train-auc:0.92174
[6101]	train-auc:0.92175
[6102]	train-auc:0.92175
[6103]	train-auc:0.92175
[6104]	train-auc:0.92176
[6105]	train-auc:0.92176
[6106]	train-auc:0.92176
[6107]	train-auc:0.92176
[6108]	train-auc:0.92177
[6109]	train-auc:0.92177
[6110]	train-auc:0.92177
[6111]	train-auc:0.92178
[6112]	train-auc:0.92178
[6113]	train-auc:0.92178
[6114]	train-auc:0.92178
[6115]	train-auc:0.92179
[6116]	train-auc:0.92179
[6117]	train-auc:0.92179
[6118]	train-auc:0.92179
[6119]	train-auc:0.92179
[6120]	train-auc:0.92180
[6121]	train-auc:0.92180
[6122]	train-auc:0.92180
